# Task 5: Vehicle Registry with Hash Function Analysis

In [ ]:
import random
import string
import sys

sys.path.append("..")

from ads import HashTableSC, Vehicle

## 1. Basic Vehicle Registry Usage

Swedish registration format: 3 letters + 2 digits + 1 letter (e.g., "ABC12D")

Note: The hash function works with any registration format since it processes all characters.

In [ ]:
# Create hashtable for vehicle registry
registry = HashTableSC[str, Vehicle](m=31)

# Create some vehicles with Swedish registration numbers
vehicles = [
    Vehicle("ABC12D", "Volvo", "XC90", 2020),
    Vehicle("XYZ78E", "Saab", "9-5", 2018),
    Vehicle("MLB84A", "Tesla", "Model 3", 2022),
    Vehicle("GHJ45F", "BMW", "X5", 2019),
    Vehicle("QWE32G", "Audi", "A4", 2021),
    Vehicle("JGB132", "Hyundai", "i40", 2014),  # Old format also works
]

# Register vehicles (using registration as key)
for v in vehicles:
    registry.put(v.registration, v)

print(f"Registered {len(registry)} vehicles")
print("\nRegistry operations:")
print(f"  Contains 'MLB84A': {registry.contains('MLB84A')}")
print(f"  Contains 'ZZZ99Z': {registry.contains('ZZZ99Z')}")

In [ ]:
# Lookup vehicle by registration
vehicle = registry.get("JGB132")
if vehicle:
    print(f"Found vehicle: {vehicle.brand} {vehicle.model} ({vehicle.year})")
    print(f"Registration: {vehicle.registration}")

In [ ]:
# List all registered vehicles
print("All registered vehicles:")
for reg in sorted(registry.keys()):
    v = registry.get(reg)
    print(f"  {reg}: {v.brand} {v.model} ({v.year})")

In [ ]:
# Remove a vehicle
removed = registry.remove("XYZ789")
print(f"Removed XYZ789: {removed}")
print(f"Registry now has {len(registry)} vehicles")
print(f"Contains 'XYZ789': {registry.contains('XYZ789')}")

## 2. Hash Function Analysis

Generate many Swedish registration numbers and analyze the hash function quality.

In [ ]:
def generate_swedish_registration() -> str:
    """Generate a random Swedish registration number.

    Format: 3 letters + 2 digits + 1 letter
    Example: ABC12D
    """
    letters1 = "".join(random.choices(string.ascii_uppercase, k=3))
    digits = "".join(random.choices(string.digits, k=2))
    letter2 = random.choice(string.ascii_uppercase)
    return f"{letters1}{digits}{letter2}"


# Test generation
print("Sample Swedish registrations:")
for _ in range(10):
    print(f"  {generate_swedish_registration()}")

In [ ]:
# Generate large dataset of vehicles
random.seed(42)  # For reproducibility

brands = ["Volvo", "Saab", "BMW", "Audi", "Mercedes", "Tesla", "Toyota", "Volkswagen"]
models = ["Sedan", "SUV", "Coupe", "Wagon", "Hatchback"]

n_vehicles = 500
table_size = 101  # Larger prime for better distribution visualization

# Create hashtable and vehicles
test_registry = HashTableSC[str, Vehicle](m=table_size)
registrations = set()

# Generate unique registrations
while len(registrations) < n_vehicles:
    registrations.add(generate_swedish_registration())

# Create and register vehicles
for reg in registrations:
    vehicle = Vehicle(
        registration=reg,
        brand=random.choice(brands),
        model=random.choice(models),
        year=random.randint(2015, 2024),
    )
    test_registry.put(reg, vehicle)

print(f"Created registry with {len(test_registry)} vehicles")
print(f"Table size: {test_registry.sz}")
print(f"Load factor (n/m): {len(test_registry) / test_registry.sz:.2f}")

### Analyze Hash Function Quality

In [ ]:
# Calculate chain lengths for each bucket
chain_lengths = [test_registry._len_chain(bucket) for bucket in test_registry.table]

# Statistics
total_items = len(test_registry)
table_size = test_registry.sz
empty_buckets = chain_lengths.count(0)
non_empty_buckets = table_size - empty_buckets
collisions = sum(1 for length in chain_lengths if length > 1)
max_chain = max(chain_lengths)
avg_chain = sum(chain_lengths) / len(chain_lengths)
expected_chain = total_items / table_size

# Quality thresholds
DISTRIBUTION_TOLERANCE = 0.5  # Acceptable difference for good distribution
MAX_CHAIN_MULTIPLIER = 2  # Maximum acceptable chain length relative to expected

print("=" * 50)
print("Hash Function Quality Analysis")
print("=" * 50)
print("\nDataset:")
print(f"  Total vehicles: {total_items}")
print(f"  Table size (m): {table_size}")
print(f"  Load factor (α = n/m): {total_items / table_size:.2f}")
print("\nBucket Distribution:")
print(f"  Empty buckets: {empty_buckets} ({empty_buckets / table_size * 100:.1f}%)")
print(
    f"  Non-empty buckets: \
        {non_empty_buckets} ({non_empty_buckets / table_size * 100:.1f}%)"
)
print(f"  Buckets with collisions: {collisions} ({collisions / table_size * 100:.1f}%)")
print("\nChain Length Statistics:")
print(f"  Minimum: {min(chain_lengths)}")
print(f"  Maximum: {max_chain}")
print(f"  Average: {avg_chain:.2f}")
print(f"  Expected (n/m): {expected_chain:.2f}")
print("\nDeviation from expected:")
print(f"  |avg - expected| = {abs(avg_chain - expected_chain):.2f}")
print(f"  Tolerance threshold = {DISTRIBUTION_TOLERANCE}")
print(f"  Max chain / expected = {max_chain / expected_chain:.2f}")
print(f"  Max chain multiplier threshold = {MAX_CHAIN_MULTIPLIER}")

### Visualize Bucket Distribution

In [ ]:
import matplotlib.pyplot as plt

# Threshold for highlighting over-loaded buckets
OVERLOAD_MULTIPLIER = 1.5  # Highlight buckets above this threshold

# Bar chart of chain lengths
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Chain length per bucket
buckets = range(len(chain_lengths))
colors = [
    "red"
    if length == 0
    else "orange"
    if length > expected_chain * OVERLOAD_MULTIPLIER
    else "skyblue"
    for length in chain_lengths
]

ax1.bar(buckets, chain_lengths, color=colors, edgecolor="black", linewidth=0.5)
ax1.axhline(
    y=expected_chain,
    color="green",
    linestyle="--",
    linewidth=2,
    label=f"Expected (n/m = {expected_chain:.2f})",
)
ax1.set_xlabel("Bucket Index")
ax1.set_ylabel("Chain Length")
ax1.set_title(f"Chain Length Distribution\n(n={total_items}, m={table_size})")
ax1.legend()
ax1.grid(axis="y", alpha=0.3)

# Right plot: Histogram of chain lengths
unique_lengths = sorted(set(chain_lengths))
length_counts = [chain_lengths.count(length) for length in unique_lengths]

ax2.bar(unique_lengths, length_counts, color="teal", edgecolor="black", width=0.8)
ax2.set_xlabel("Chain Length")
ax2.set_ylabel("Number of Buckets")
ax2.set_title("Frequency of Chain Lengths")
ax2.grid(axis="y", alpha=0.3)

# Add value labels on bars
for length, count in zip(unique_lengths, length_counts, strict=False):
    ax2.text(length, count, str(count), ha="center", va="bottom")

plt.tight_layout()
plt.show()

## 3. Hash Function Implementation

The Vehicle class uses a custom `__hash__()` method:

```python
def __hash__(self) -> int:
    hv = 17
    for char in self.registration:
        hv = 31 * hv + ord(char)
    return hv
```

This is a polynomial rolling hash that:
- Starts with prime number 17
- Multiplies by prime 31 for each character
- Adds each character's ordinal value
- Makes position matter (order-sensitive)

## 4. Hash Function Comparison

Compare polynomial rolling hash with simple addition hash:

In [ ]:
def simple_hash(reg: str) -> int:
    """Naive hash: just sum character values."""
    return sum(ord(c) for c in reg)


def good_hash(reg: str) -> int:
    """Polynomial rolling hash with primes."""
    hv = 17
    for char in reg:
        hv = 31 * hv + ord(char)
    return hv


# Test with sample registrations (including anagrams)
test_regs = ["ABC12D", "ABC21D", "BAC12D", "CBA21D", "JGB132"]

print("Hash function comparison:\n")
print(
    f"{'Registration':<12} {'Simple Hash':<15} {'Simple % 101':<15} {'Good % 101':<15}"
)
print("-" * 65)
for reg in test_regs:
    simple = simple_hash(reg)
    good = good_hash(reg)
    print(f"{reg:<12} {simple:<15} {simple % 101:<15} {good % 101:<15}")

print("\n" + "=" * 65)
print("Anagram collision test:")
print("=" * 65)
print("\nSimple hash (BAD - position doesn't matter):")
print(f"  'ABC12D' -> {simple_hash('ABC12D')}")
print(f"  'BAC12D' -> {simple_hash('BAC12D')}")
print(f"  Same hash? {simple_hash('ABC12D') == simple_hash('BAC12D')} ✗ COLLISION!")

print("\nPolynomial hash (GOOD - position matters):")
print(f"  'ABC12D' -> {good_hash('ABC12D')}")
print(f"  'BAC12D' -> {good_hash('BAC12D')}")
print(f"  Same hash? {good_hash('ABC12D') == good_hash('BAC12D')} ✓ NO COLLISION!")

print("\nAfter modulo 101:")
print(f"  'ABC12D' % 101 = {good_hash('ABC12D') % 101}")
print(f"  'BAC12D' % 101 = {good_hash('BAC12D') % 101}")
print(
    f"  Different buckets: {good_hash('ABC12D') % 101 != good_hash('BAC12D') % 101} ✓"
)